# Programme data

This notebook prepares data for the Programme theme page.

In [ ]:
import json
from datetime import date

from utils.themes.programme import Programme
import petl as etl

In [ ]:
from utils.paths import SITE

EVENTS = SITE / 'themes/programme/_data/events'
EVENTS.mkdir(exist_ok=True, parents=True)

## Schedule events

Aggregation definition

In [ ]:
event_aggregation = {
    'Records': ('row', set),
    'Events': ('Event Count', sum),
    'Date From': ('Start Date', min),
    'Date To': ('End Date', max),
}

Read events and break down into chunks broken by month boundaries. This deals with multi-day events which cover more than a single month.

As an example:

| Start -> End | Chunks |
|--------------|--------|
| 15 Jan -> 18 Jan | 1 chunk (15 Jan -> 18 Jan) |
| 15 Jan -> 18 Feb | 2 chunks (15 Jan -> 31 Jan, 1 Feb -> 18 Feb) |
| 15 Jan -> 18 Mar | 3 chunks (15 Jan -> 31 Jan, 1 Feb -> 28 Feb, 1 Mar -> 18 Mar) |

Each chunk is then rendered as a separate row in the table with the `rowmapmany` method.

In [ ]:
monthly_event_reports = (
    Programme.event_reports
    .addfield('Month', lambda r: r.event_date.replace(day=1))
    .aggregate(['Project Name', 'project_id', 'Month'], { 'event_reports': (len), 'event_reports_audience': ('audience', sum) })
)

In [ ]:
monthly_scheduled_events = (
    Programme.events
    .convert(
        'Event Count',
        lambda _, r: ((date.today() - r['Start Date']).days + 1),
        where=lambda r: (r['End Date'] > date.today()),
        pass_row=True
    )
    .aggregate(['Project Name', 'project_id', 'Month'], { 'scheduled_events': ('Event Count', sum) })
)

In [ ]:
import petl as etl
from utils.paths import PUBLISHED

monthly_manual_events = (
    etl.fromcsv(PUBLISHED / 'manual/manual-events.csv')
    .replace('Events', '', 1)
    .replace('Audience', '', 0)
    .convertnumbers()
    .selectne('Exclude from events count', 'True')
    .addfield('Month', lambda r: date.fromisoformat(r.Date).replace(day=1))
    .aggregate(['Project', 'Month'], {
        'manual_events': ('Events', sum),
        'manual_audience': ('Audience', sum),
    })
    .rename({
        'Project': 'Project Name'
    })
    .addfield('project_id')
)


In [ ]:
monthly_events = (
    monthly_event_reports
    .outerjoin(monthly_scheduled_events)
    .outerjoin(monthly_manual_events)
    .addfield('events', lambda r: r.event_reports or r.scheduled_events or r.manual_events, index=3)
    .addfield('audience', lambda r: r.event_reports_audience or r.manual_audience, index=4)
)

Create an aggregate by month of the events

In [ ]:
(
    monthly_events
    .aggregate('Month', {
        'Events': ('events', sum),
        # 'Records': (len),
    })
    .convert('Month', lambda f: f.isoformat())
    .tocsv(EVENTS / 'total_by_month.csv')
)

Aggregate by Project and by Month, and convert months to columns

In [ ]:
(
    monthly_events
    .aggregate(['Project Name', 'Month'], sum, 'events')
    .recast(key='Project Name', variablefield='Month', missing=0)
    .tocsv(EVENTS / 'monthly_by_project.csv')
)

Aggregate by Project and by Month, and convert projects to columns

In [ ]:
(
    monthly_events
    .aggregate(['Project Name', 'Month'], sum, 'events')
    .recast(key='Month', variablefield='Project Name', missing=0)
    .tocsv(EVENTS / 'monthly_breakdown.csv')
)

## Project summaries

In [ ]:
project_breakdown = (
    monthly_events
    .melt(variables=['events', 'event_reports', 'scheduled_events', 'manual_events', 'audience', 'event_reports_audience', 'manual_audience'])
    .selectnotnone('value')
    .aggregate(['Project Name', 'project_id', 'variable'], sum, 'value')
    .recast()
    .join(Programme.projects.cutout('Project Name'), lkey='project_id', rkey='id')
)

Create a project breakdown

In [ ]:
with open(EVENTS / 'by_project.json', 'w') as f:
    json.dump(
        dict(
            project_breakdown
            .addfield('Details', lambda r: {
                # 'records': r.Records,
                'events': r.events,
                'eventReports': r.event_reports,
                'scheduledEvents': r.scheduled_events,
                'manual_events': r.manual_events,
                'audience': r.audience,
                'event_reports_audience': r.event_reports_audience,
                'manual_audience': r.manual_audience,
                'evaluationCategory': r['Evaluation Category'],
                'programmeCategory': r['Programme Category'],
                'earliestDate': r['Start Date'].isoformat() if r['Start Date'] else None,
                'latestDate': r['End Date'].isoformat() if r['End Date'] else None,
            })
            .cut('Project Name', 'Details')
            .sort('Project Name')
            .records()
        ),
        f,
        indent=2,
    )

Create a summary file

In [ ]:
with open(EVENTS / 'summary.json', 'w') as f:
    json.dump(
        {
            'total': sum(monthly_events.values('events')),
            'excluded': dict(Programme.excluded_events.aggregate('Validation', len).records()),
            'date': {
                'earliest': min(Programme.events.values('Start Date')).isoformat(),
                'latest': max(Programme.events.values('End Date')).isoformat(),
            }
        },
        f,
        indent=2,
    )

## Venues

In [ ]:
education_settings = (
    Programme.venues
    .selectcontains('Org/Venue Type', 'Education Setting')
)

In [ ]:
loading_bay = (
    Programme.venues
    .selectcontains('Organisation &/or Venue Name', 'Loading Bay')
    .addfield('event_report_count', lambda r: len(r['Event Reports']))
    .aggregate(['Organisation &/or Venue Name', 'id'], sum, 'event_report_count')
    .cutout('id')
)
loading_bay

In [ ]:
beacon = (
    Programme.venues
    .selectcontains('Organisation &/or Venue Name', 'Beacon - ')
    .addfield('event_report_count', lambda r: len(r['Event Reports'] or []))
    .aggregate(['Organisation &/or Venue Name', 'id'], sum, 'event_report_count')
    .cutout('id')
)
beacon

In [ ]:
json.dump(
    {
        'loading_bay': dict(loading_bay.records()),
        'beacon': dict(beacon.records()),
    },
    open(EVENTS / 'by_venue.json', 'w')
)